In [10]:
from __future__ import print_function
import sys
print (sys.executable)

/Users/stephanieliu/miniconda3/bin/python


In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import scipy.io as sio

import train_batch as train

/Users/stephanieliu/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


#### Define the blocks
Always remember the mental model:
input => representation learning => loss => optimizer

Step 1: define input (and output, because during the training we need the ground truth as reference)

In [4]:
tf.reset_default_graph()
# [1] Input
x = tf.placeholder(tf.float32, [None, train.INTERVAL_SIZE])
# y_ holds the true output, 
# we use this placeholder to pass the true recognition result to teach the network
y_ = tf.placeholder(tf.int32, [None])

Step 2: Representation learning (the network that reveals the information contained in the image). Here we use 1 layer of fully connected network

In [5]:
# [2] network, first layer
W = tf.Variable(tf.zeros([1024, 76])) # can use 76 for simplicity
b = tf.Variable(tf.zeros([76]))
O = tf.matmul(x, W) + b

# Second layer
W_2 = tf.Variable(tf.zeros([76, 76])) # 76 x 76 

# while the layer is not the last layer, we can have any number, bottle neck layer < 76, extract information > 76

Step 3: loss function, to define what's a good prediction 

In [6]:
# [3] loss, this loss function below outputs a scalar to show who close the network's guess
# O can represent the true recognition result y_. 
# Here O[j] shows a rating of how likely x is digit j, we will show that later
loss_all = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=O, labels=y_)
loss = tf.reduce_mean(loss_all)

Step 4: Optimizer, strategy for training

In [7]:
# [4] Optimizer
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
# [optional, compute accuracy so that we can know how well we are doing while training]
prediction = tf.argmax(tf.nn.softmax(O), 1, output_type=tf.int32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(prediction, y_), tf.float32))

#### Training
Call "train_step" a number of time; you can call this block multiple times until it reaches a good accuracy

In [9]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# Randomly draw a file, for that file, randomly draw an interval of 1024 and find corresponding label
# For now, read all data into memory - HAVE QUESTION ABOUT THIS

#training, stop when objective value (o) is small enough
for step in range(1000):
    batch_xs, batch_ys = train.next_batch(64)
    _, v_loss = sess.run([train_step, loss], feed_dict={x: batch_xs, y_: batch_ys})
    if (step % 100 == 99):
        batch_xs, batch_ys = train.next_batch(512) #change this to test batch
        v_acc, v_loss = sess.run([accuracy, loss], feed_dict={x: batch_xs, y_: batch_ys})
        # print the loss every 100 interations
        print('accuracy=', v_acc, 'loss=', v_loss)

accuracy= 0.330078 loss= 3.24905
accuracy= 0.294922 loss= 3.25625
accuracy= 0.318359 loss= 3.1859
accuracy= 0.361328 loss= 3.03669
accuracy= 0.316406 loss= 3.13469
accuracy= 0.365234 loss= 2.96405
accuracy= 0.341797 loss= 3.05914
accuracy= 0.300781 loss= 3.20376
accuracy= 0.365234 loss= 2.97261
accuracy= 0.349609 loss= 3.06508
